# prednet

https://github.com/leido/pytorch-prednet

$n$フレームの学習から$n+1$フレームを予測している

96フレームの学習で次の24フレームを学習するようにしたい。

ついでに欠損画像の確認もしている。

In [1]:
import glob
import gzip
import os
import shutil
import warnings
from datetime import datetime as dt
from datetime import timedelta
from PIL import Image
import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.basemap import Basemap

import torch
import torch.utils.data as data

from torch.utils.data import DataLoader
import os

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader

# from kitti_data import KITTI
from prednet import PredNet

from debug import info

%matplotlib inline
warnings.filterwarnings(
    'ignore', category=matplotlib.MatplotlibDeprecationWarning)

In [2]:
img_list = glob.glob('../data/train/sat/*/*')

img_df = pd.DataFrame({'img_path': img_list})
img_df['img_path'] = img_df['img_path'].str.replace('\\', '/')
img_df['img_file_name'] = img_df['img_path'].apply(lambda x: x.split('/')[-1])
img_df['img_date'] = img_df['img_file_name'].apply(lambda x: x.split('.')[0])
img_df['img_date'] = pd.to_datetime(img_df['img_date'])
img_df.index = img_df['img_date']
start_at, end_at = img_df['img_date'].min(), img_df['img_date'].max()
img_df = img_df.reindex(pd.date_range(start_at, end_at, freq="H"))
print(len(img_df))

img_not_exist_df = img_df[img_df['img_date'].isna()]
if len(img_df[img_df['img_date'].isna()]) == 0:
    img_not_exist_df = img_df[img_df['img_file_name'].str.contains('dummy')]
    img_df = img_df[~img_df['img_file_name'].str.contains('dummy')]
img_not_exist_df = img_not_exist_df.assign(date_diff=img_not_exist_df.index.to_series(
    keep_tz=True).diff().astype('timedelta64[h]')).fillna(0)
img_not_exist_df['img_date'] = img_not_exist_df.index.values
img_not_exist_df['img_date'] = img_not_exist_df['img_date'].dt.strftime(
    '%Y-%m-%d-%H-%M')
img_not_exist_df['img_file_name'] = img_not_exist_df['img_date'] + '.dummy.fv.png'
img_not_exist_df['img_year'] = img_not_exist_df['img_file_name'].apply(
    lambda x: x.split('-')[0])
img_not_exist_df['img_month'] = img_not_exist_df['img_file_name'].apply(
    lambda x: x.split('-')[1])
img_not_exist_df['img_day'] = img_not_exist_df['img_file_name'].apply(
    lambda x: x.split('-')[2])
img_not_exist_df['img_yyyy-mm-dd'] = img_not_exist_df['img_year'] + \
    '-'+img_not_exist_df['img_month'] + '-' + img_not_exist_df['img_day']
img_not_exist_df['img_path'] = '../data/train/sat/' + \
    img_not_exist_df['img_yyyy-mm-dd'] + '/' + img_not_exist_df['img_file_name']
img_not_exist_df = img_not_exist_df[['img_path', 'img_file_name', 'date_diff']]
img_not_exist_df = img_not_exist_df.assign(file_not_exist=1)
display(img_not_exist_df.head())
print('欠落データ数: {}'.format(len(img_not_exist_df)))

img_df = img_df.drop('img_date', axis=1)
img_df = img_df.dropna()
img_df = img_df.assign(date_diff=0)
img_df = img_df.assign(file_not_exist=0)

img_df = pd.concat([img_df, img_not_exist_df]).sort_index()
len(img_df)

17543


,img_path,img_file_name,date_diff,file_not_exist
2016-01-22 21:00:00+00:00,../data/train/sat/2016-01-22/2016-01-22-21-00....,2016-01-22-21-00.dummy.fv.png,0.0,1
2016-02-07 21:00:00+00:00,../data/train/sat/2016-02-07/2016-02-07-21-00....,2016-02-07-21-00.dummy.fv.png,384.0,1
2016-02-15 15:00:00+00:00,../data/train/sat/2016-02-15/2016-02-15-15-00....,2016-02-15-15-00.dummy.fv.png,186.0,1
2016-02-15 16:00:00+00:00,../data/train/sat/2016-02-15/2016-02-15-16-00....,2016-02-15-16-00.dummy.fv.png,1.0,1
2016-02-15 17:00:00+00:00,../data/train/sat/2016-02-15/2016-02-15-17-00....,2016-02-15-17-00.dummy.fv.png,1.0,1


欠落データ数: 56


17543

In [3]:
class KITTI(data.Dataset):
    def __init__(self, X, nt):
        self.X = X        
        self.nt = nt
        cur_loc = 0
        possible_starts = []
        while cur_loc < self.X.shape[0] - self.nt + 1:
            possible_starts.append(cur_loc)
            cur_loc += self.nt
        self.possible_starts = possible_starts

    def __getitem__(self, index):
        loc = self.possible_starts[index]
        return self.X[loc:loc+self.nt]

    def __len__(self):
        return len(self.possible_starts)

In [4]:
img_df

,img_path,img_file_name,date_diff,file_not_exist
2016-01-01 01:00:00+00:00,../data/train/sat/2016-01-01/2016-01-01-01-00....,2016-01-01-01-00.fv.png,0.0,0
2016-01-01 02:00:00+00:00,../data/train/sat/2016-01-01/2016-01-01-02-00....,2016-01-01-02-00.fv.png,0.0,0
2016-01-01 03:00:00+00:00,../data/train/sat/2016-01-01/2016-01-01-03-00....,2016-01-01-03-00.fv.png,0.0,0
2016-01-01 04:00:00+00:00,../data/train/sat/2016-01-01/2016-01-01-04-00....,2016-01-01-04-00.fv.png,0.0,0
2016-01-01 05:00:00+00:00,../data/train/sat/2016-01-01/2016-01-01-05-00....,2016-01-01-05-00.fv.png,0.0,0
...,...,...,...,...
2017-12-31 19:00:00+00:00,../data/train/sat/2017-12-31/2017-12-31-19-00....,2017-12-31-19-00.fv.png,0.0,0
2017-12-31 20:00:00+00:00,../data/train/sat/2017-12-31/2017-12-31-20-00....,2017-12-31-20-00.fv.png,0.0,0
2017-12-31 21:00:00+00:00,../data/train/sat/2017-12-31/2017-12-31-21-00....,2017-12-31-21-00.fv.png,0.0,0
2017-12-31 22:00:00+00:00,../data/train/sat/2017-12-31/2017-12-31-22-00....,2017-12-31-22-00.fv.png,0.0,0


In [5]:
from PIL import Image

width = 512//4  # width of input image
height = 672//4  # height of imput image
# n = 300

X = np.array([])
for path in img_df['img_path']:
    img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, dsize=(width, height), interpolation=cv2.INTER_CUBIC)
    X = np.append(X, img)
X = X.reshape(len(img_df), height, width, 1)
train_X = KITTI(X, nt=10)
# for i, train in enumerate(train_X):
#     print(train.shape)
#     plt.figure(figsize=(15,30))
#     for b in range(5):
#         plt.subplot(1,5,b+1)
#         plt.imshow(train[b].reshape(height, width), 'gray')
#         plt.tick_params(labelbottom=False,
#         labelleft=False,
#         labelright=False,
#         labeltop=False)
#     plt.show()

In [6]:
train_loader = DataLoader(train_X, batch_size=8, shuffle=True)

In [7]:
num_epochs = 150
batch_size = 8
A_channels = (1, 48, 96, 192)
R_channels = (1, 48, 96, 192)
lr = 0.001  # if epoch < 75 else 0.0001
nt = 10  # num of time steps

layer_loss_weights = Variable(
    torch.FloatTensor([[1.], [0.], [0.], [0.]]).cuda())
time_loss_weights = 1./(nt - 1) * torch.ones(nt, 1)
time_loss_weights[0] = 0
time_loss_weights = Variable(time_loss_weights.cuda())

In [8]:
model = PredNet(R_channels, A_channels, output_mode='error')
if torch.cuda.is_available():
    print('Using GPU.')
    model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def lr_scheduler(optimizer, epoch):
    if epoch < num_epochs // 2:
        return optimizer
    else:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.0001
        return optimizer

Using GPU.


In [9]:

for epoch in range(num_epochs):
    optimizer = lr_scheduler(optimizer, epoch)
    for i, inputs in enumerate(train_loader):
        # batch x time_steps x channel x width x height
        inputs = inputs.permute(0, 1, 4, 2, 3)
        inputs = Variable(inputs.cuda())
        errors = model(inputs)  # batch x n_layers x nt
        loc_batch = errors.size(0)
        # batch*n_layers x 1
        errors = torch.mm(errors.view(-1, nt), time_loss_weights)
        errors = torch.mm(errors.view(loc_batch, -1), layer_loss_weights)
        errors = torch.mean(errors)

        optimizer.zero_grad()

        errors.backward()

        optimizer.step()
        # print(kitti_train)
        if i % 10 == 0:
            print('Epoch: {}/{}, step: {}/{}, errors: {}'.format(epoch,
                                                                 num_epochs, i, len(train_X)//batch_size, errors.data))

torch.save(model.state_dict(), 'training.pt')

c:\users\yui\documents\kaggle\weather_challenge\.venv\lib\site-packages\torch\nn\functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
c:\users\yui\documents\kaggle\weather_challenge\.venv\lib\site-packages\torch\nn\functional.py:1339: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch: 0/150, step: 0/219, errors: 49.170204162597656
Epoch: 0/150, step: 10/219, errors: 47.4660758972168
Epoch: 0/150, step: 20/219, errors: 52.47607421875
Epoch: 0/150, step: 30/219, errors: 47.735389709472656
Epoch: 0/150, step: 40/219, errors: 49.434654235839844
Epoch: 0/150, step: 50/219, errors: 44.73407745361328
Epoch: 0/150, step: 60/219, errors: 49.38462829589844
Epoch: 0/150, step: 70/219, errors: 47.587921142578125
Epoch: 0/150, step: 80/219, errors: 49.62511444091797
Epoch: 0/150, step: 90/219, errors: 45.08789825439453
Epoch: 0/150, step: 100/219, errors: 47.84741973876953
Epoch: 0/150, step: 110/219, errors: 45.724693298339844
Epoch: 0/150, step: 120/219, errors: 47.27442169189453
Epoch: 0/150, step: 130/219, errors: 46.840049743652344
Epoch: 0/150, step: 140/219, errors: 47.65607452392578
Epoch: 0/150, step: 150/219, errors: 49.20582962036133
Epoch: 0/150, step: 160/219, errors: 44.849517822265625
Epoch: 0/150, step: 170/219, errors: 48.99589538574219
Epoch: 0/150, step

Epoch: 6/150, step: 180/219, errors: 40.665748596191406
Epoch: 6/150, step: 190/219, errors: 36.496177673339844
Epoch: 6/150, step: 200/219, errors: 39.817893981933594
Epoch: 6/150, step: 210/219, errors: 41.730777740478516
Epoch: 7/150, step: 0/219, errors: 37.47598648071289
Epoch: 7/150, step: 10/219, errors: 45.02410125732422
Epoch: 7/150, step: 20/219, errors: 37.70600128173828
Epoch: 7/150, step: 30/219, errors: 45.73075866699219
Epoch: 7/150, step: 40/219, errors: 44.04561996459961
Epoch: 7/150, step: 50/219, errors: 39.34638977050781
Epoch: 7/150, step: 60/219, errors: 44.79493713378906
Epoch: 7/150, step: 70/219, errors: 37.699031829833984
Epoch: 7/150, step: 80/219, errors: 38.233428955078125
Epoch: 7/150, step: 90/219, errors: 39.52882766723633
Epoch: 7/150, step: 100/219, errors: 39.45600509643555
Epoch: 7/150, step: 110/219, errors: 41.69349670410156
Epoch: 7/150, step: 120/219, errors: 44.567596435546875
Epoch: 7/150, step: 130/219, errors: 37.356014251708984
Epoch: 7/150,

Epoch: 13/150, step: 130/219, errors: 30.400928497314453
Epoch: 13/150, step: 140/219, errors: 34.29481887817383
Epoch: 13/150, step: 150/219, errors: 35.73202896118164
Epoch: 13/150, step: 160/219, errors: 30.760114669799805
Epoch: 13/150, step: 170/219, errors: 37.16690444946289
Epoch: 13/150, step: 180/219, errors: 35.15233612060547
Epoch: 13/150, step: 190/219, errors: 29.029199600219727
Epoch: 13/150, step: 200/219, errors: 36.451507568359375
Epoch: 13/150, step: 210/219, errors: 36.23777770996094
Epoch: 14/150, step: 0/219, errors: 37.612571716308594
Epoch: 14/150, step: 10/219, errors: 36.514625549316406
Epoch: 14/150, step: 20/219, errors: 29.66339874267578
Epoch: 14/150, step: 30/219, errors: 34.432655334472656
Epoch: 14/150, step: 40/219, errors: 30.488754272460938
Epoch: 14/150, step: 50/219, errors: 32.91194534301758
Epoch: 14/150, step: 60/219, errors: 33.28495407104492
Epoch: 14/150, step: 70/219, errors: 29.595327377319336
Epoch: 14/150, step: 80/219, errors: 31.00829887

Epoch: 20/150, step: 60/219, errors: 29.491470336914062
Epoch: 20/150, step: 70/219, errors: 27.983421325683594
Epoch: 20/150, step: 80/219, errors: 28.019512176513672
Epoch: 20/150, step: 90/219, errors: 28.592906951904297
Epoch: 20/150, step: 100/219, errors: 28.934093475341797
Epoch: 20/150, step: 110/219, errors: 28.357486724853516
Epoch: 20/150, step: 120/219, errors: 25.420188903808594
Epoch: 20/150, step: 130/219, errors: 28.669231414794922
Epoch: 20/150, step: 140/219, errors: 28.437114715576172
Epoch: 20/150, step: 150/219, errors: 31.046300888061523
Epoch: 20/150, step: 160/219, errors: 27.10043716430664
Epoch: 20/150, step: 170/219, errors: 28.298385620117188
Epoch: 20/150, step: 180/219, errors: 30.852890014648438
Epoch: 20/150, step: 190/219, errors: 26.307418823242188
Epoch: 20/150, step: 200/219, errors: 25.863771438598633
Epoch: 20/150, step: 210/219, errors: 28.573999404907227
Epoch: 21/150, step: 0/219, errors: 30.400222778320312
Epoch: 21/150, step: 10/219, errors: 2

...

Epoch: 120/150, step: 60/219, errors: 8.785284042358398
Epoch: 120/150, step: 70/219, errors: 9.175054550170898
Epoch: 120/150, step: 80/219, errors: 6.546146392822266
Epoch: 120/150, step: 90/219, errors: 7.710636138916016
Epoch: 120/150, step: 100/219, errors: 6.913876533508301
Epoch: 120/150, step: 110/219, errors: 7.988117694854736
Epoch: 120/150, step: 120/219, errors: 7.9917778968811035
Epoch: 120/150, step: 130/219, errors: 8.545328140258789
Epoch: 120/150, step: 140/219, errors: 9.005058288574219
Epoch: 120/150, step: 150/219, errors: 8.35085391998291
Epoch: 120/150, step: 160/219, errors: 8.463997840881348
Epoch: 120/150, step: 170/219, errors: 8.606974601745605
Epoch: 120/150, step: 180/219, errors: 8.289284706115723
Epoch: 120/150, step: 190/219, errors: 8.210549354553223
Epoch: 120/150, step: 200/219, errors: 7.352560043334961
Epoch: 120/150, step: 210/219, errors: 8.274497985839844
Epoch: 121/150, step: 0/219, errors: 8.373811721801758
Epoch: 121/150, step: 10/219, errors:

Epoch: 126/150, step: 200/219, errors: 9.751245498657227
Epoch: 126/150, step: 210/219, errors: 7.7857513427734375
Epoch: 127/150, step: 0/219, errors: 7.567785263061523
Epoch: 127/150, step: 10/219, errors: 7.571706771850586
Epoch: 127/150, step: 20/219, errors: 8.77346134185791
Epoch: 127/150, step: 30/219, errors: 8.414253234863281
Epoch: 127/150, step: 40/219, errors: 7.993257999420166
Epoch: 127/150, step: 50/219, errors: 7.167362689971924
Epoch: 127/150, step: 60/219, errors: 8.115299224853516
Epoch: 127/150, step: 70/219, errors: 10.366388320922852
Epoch: 127/150, step: 80/219, errors: 9.575078010559082
Epoch: 127/150, step: 90/219, errors: 7.701704025268555
Epoch: 127/150, step: 100/219, errors: 8.846687316894531
Epoch: 127/150, step: 110/219, errors: 9.984756469726562
Epoch: 127/150, step: 120/219, errors: 8.493202209472656
Epoch: 127/150, step: 130/219, errors: 6.9453043937683105
Epoch: 127/150, step: 140/219, errors: 6.978840351104736
Epoch: 127/150, step: 150/219, errors: 8

Epoch: 133/150, step: 120/219, errors: 8.2510986328125
Epoch: 133/150, step: 130/219, errors: 7.355499744415283
Epoch: 133/150, step: 140/219, errors: 8.651786804199219
Epoch: 133/150, step: 150/219, errors: 9.225173950195312
Epoch: 133/150, step: 160/219, errors: 9.792767524719238
Epoch: 133/150, step: 170/219, errors: 8.305249214172363
Epoch: 133/150, step: 180/219, errors: 9.00960922241211
Epoch: 133/150, step: 190/219, errors: 7.667632102966309
Epoch: 133/150, step: 200/219, errors: 7.8022565841674805
Epoch: 133/150, step: 210/219, errors: 7.694891452789307
Epoch: 134/150, step: 0/219, errors: 9.313249588012695
Epoch: 134/150, step: 10/219, errors: 8.9403715133667
Epoch: 134/150, step: 20/219, errors: 7.994405746459961
Epoch: 134/150, step: 30/219, errors: 7.479485988616943
Epoch: 134/150, step: 40/219, errors: 8.587149620056152
Epoch: 134/150, step: 50/219, errors: 7.557323455810547
Epoch: 134/150, step: 60/219, errors: 7.564770698547363
Epoch: 134/150, step: 70/219, errors: 7.654

Epoch: 140/150, step: 40/219, errors: 6.8444600105285645
Epoch: 140/150, step: 50/219, errors: 6.768383026123047
Epoch: 140/150, step: 60/219, errors: 7.695583343505859
Epoch: 140/150, step: 70/219, errors: 8.37348461151123
Epoch: 140/150, step: 80/219, errors: 9.190762519836426
Epoch: 140/150, step: 90/219, errors: 8.993486404418945
Epoch: 140/150, step: 100/219, errors: 9.33598518371582
Epoch: 140/150, step: 110/219, errors: 8.133487701416016
Epoch: 140/150, step: 120/219, errors: 6.794862270355225
Epoch: 140/150, step: 130/219, errors: 6.842248916625977
Epoch: 140/150, step: 140/219, errors: 8.67405891418457
Epoch: 140/150, step: 150/219, errors: 8.338614463806152
Epoch: 140/150, step: 160/219, errors: 8.909784317016602
Epoch: 140/150, step: 170/219, errors: 7.832911491394043
Epoch: 140/150, step: 180/219, errors: 6.629006862640381
Epoch: 140/150, step: 190/219, errors: 7.543693542480469
Epoch: 140/150, step: 200/219, errors: 8.454164505004883
Epoch: 140/150, step: 210/219, errors: 

Epoch: 146/150, step: 180/219, errors: 9.077499389648438
Epoch: 146/150, step: 190/219, errors: 6.5851969718933105
Epoch: 146/150, step: 200/219, errors: 6.427874565124512
Epoch: 146/150, step: 210/219, errors: 6.519326686859131
Epoch: 147/150, step: 0/219, errors: 8.215818405151367
Epoch: 147/150, step: 10/219, errors: 7.758201599121094
Epoch: 147/150, step: 20/219, errors: 6.6811957359313965
Epoch: 147/150, step: 30/219, errors: 7.152915954589844
Epoch: 147/150, step: 40/219, errors: 7.739977836608887
Epoch: 147/150, step: 50/219, errors: 10.21950912475586
Epoch: 147/150, step: 60/219, errors: 9.131327629089355
Epoch: 147/150, step: 70/219, errors: 7.759163856506348
Epoch: 147/150, step: 80/219, errors: 7.453718185424805
Epoch: 147/150, step: 90/219, errors: 6.712884902954102
Epoch: 147/150, step: 100/219, errors: 8.143181800842285
Epoch: 147/150, step: 110/219, errors: 8.224433898925781
Epoch: 147/150, step: 120/219, errors: 7.874753475189209
Epoch: 147/150, step: 130/219, errors: 7

In [10]:
model = PredNet(R_channels, A_channels, output_mode='prediction')
model.load_state_dict(torch.load('training.pt'))

<All keys matched successfully>

In [11]:
if torch.cuda.is_available():
    print('Using GPU.')
    model.cuda()

Using GPU.


In [14]:
def save_image(tensor, filename, nrow=8, padding=2,
               normalize=False, range=None, scale_each=False, pad_value=0):
    from PIL import Image
    im = Image.fromarray(np.rollaxis(tensor.numpy(), 0, 3))
    im.save(filename)

import torchvision

for i, inputs in enumerate(train_loader):
    # batch x time_steps x channel x width x height
    inputs = inputs.permute(0, 1, 4, 2, 3)
    inputs = Variable(inputs.cuda())
    origin = inputs.data.cpu().byte()[:, nt-1]
    print('origin:')
    print(type(origin))
    print(origin.size())

    print('predicted:')
    pred = model(inputs)
    pred = pred.data.cpu().byte()
    print(type(pred))
    print(pred.size())
    origin = torchvision.utils.make_grid(origin, nrow=4)
    pred = torchvision.utils.make_grid(pred, nrow=4)
    save_image(origin, 'origin.jpg')
    save_image(pred, 'predicted.jpg')
    break


origin:
<class 'torch.Tensor'>
torch.Size([8, 1, 168, 128])
predicted:
<class 'torch.Tensor'>
torch.Size([8, 1, 168, 128])
